We have a 3rd party lead generator. We pay when lead reaches the following status:
Lead status = Converted or
Lead status = Retainer Received or
ROSU (Referred out, Signed up) = 1

Our CFO needs the billing summed by venue, litigation/casetype, and month.

The amount we pay is based on the litigation and some specific casetypes:

personal injury (default) = 1000
elmiron = 2400
tmobile data breach = 20
Philips CPAP = 500
workers comp = 250
sunscreen = 1000
paraquat = 2000
social security = 100

Our attribution was incorrectly tracked until Sep 1st of 2021. Before this date, we were manually matching phone numbers that came through their system to calls that we received and paying for all converted leads from those numbers. Starting Sep 1st, we have our attribution correct, so we stopped paying for all leads and only the ones that came from Ask.Law.

One of our concerns is that a potential client may have filled out a form on the AskLaw website and then called to complete the consultation. This means that their record will show up twice in our dataset. We need to clean the duplicates from our data so we don't pay twice for these leads.

In joining the CTEs for our baseline intakes (based on marketing source and subsource) with our CTEs that are based on pulling records based on incoming phone number, we have to be careful about how we drop duplicates. In this table we don't have phone numbers. But because we may have them in the actual dashboard we need to be sure that we drop duplicates on the specific column that will drop any subsequent intake record generated - regardless of whether that row has a phone number associated or not.

Our tasks, therefore:

    remove dupes
    filter for converted leads
    filter for post-Aug leads
    add value column for lit/casetype
    output a pretty pivot table for the CFO



In [26]:
import datetime as dt
import pandas as pd
import numpy as np

In [27]:
# Load data, set created_date to datetime datatype
data = pd.read_csv("randomized_vendor_data_2021-11-29.csv", parse_dates=['created_date'])

# Removed excess index column
data = data.drop(columns='Unnamed: 0', axis=1)

# Filter for Converted or Retainer Received OR ROSU
retained_intakes = data[
    (data.status.str.contains("Converted|Retainer Received", na=False)) | data.rosu == 1
]

# Removed duplicate intake IDs
retained_intakes = retained_intakes.sort_values(by=['created_date', 'intake'], ascending=True)
retained_intakes = retained_intakes.drop_duplicates(subset='intake', keep='first')

In [28]:
# This is a list created to the amount payed based on litigation and specific casetypes.

# personal injury (default) = 1000
# elmiron = 2400
# tmobile data breach = 20
# Philips CPAP = 500
# workers comp = 250
# sunscreen = 1000
# paraquat = 2000
# social security = 100

In [29]:
# Creating a value column with the personal injury being the defualt value which is 1000
retained_intakes['value'] = 1000

In [30]:
# checking the values printed for billing
retained_intakes[['litigation', 'casetype', 'value']].value_counts()

litigation            casetype                  value
Mass Arbitration      T-Mobile Data Breach      1000     1613
Personal Injury       Automobile Accident       1000      111
Premises Liability    Slip and Fall             1000       30
Workers Compensation  Workers Compensation      1000       28
Mass Tort             Phillips Respironic CPAP  1000       23
Premises Liability    General Injury            1000       15
Mass Tort             Elmiron                   1000        8
Product Liability     Product Liability         1000        6
Mass Tort             Paraquat                  1000        5
                      Sunscreen                 1000        2
Social Security       Concurrent                1000        2
                      SSI                       1000        1
                      SSDI                      1000        1
Insurance Dispute     Homeowners Insurance      1000        1
Premises Liability    Negligent Security        1000        1
Medical Malpract

In [31]:
# Changing the values for each litigation and casetype
retained_intakes.loc[retained_intakes.casetype.str.contains('elmiron', case=False, na=False), 'value'] = 2400
retained_intakes.loc[retained_intakes.casetype.str.contains('T-Mobile Data Breach', case=False, na=False), 'value'] = 20
retained_intakes.loc[retained_intakes.casetype.str.contains('Phillips Respironic CPAP', case=False, na=False), 'value'] = 500
retained_intakes.loc[retained_intakes.casetype.str.contains('Workers Compensation ', case=False, na=False), 'value'] = 250
retained_intakes.loc[retained_intakes.casetype.str.contains(' Paraquat', case=False, na=False), 'value'] = 2000
retained_intakes.loc[retained_intakes.casetype.str.contains('Concurrent ', case=False, na=False), 'value'] = 100

In [32]:
# filtering the rows based on the casetype
retained_intakes.loc[retained_intakes.casetype.str.contains('elmiron', case=False, na=False)]

,created_date,intake,litigation,casetype,input_channel,status,rosu,marketing_source,marketing_subsource,venue,value
3412,2021-09-25,INT-11909424,Mass Tort,Elmiron,infra-intakes,Converted,0,ForThePeople-com-INTERNET,NaN,Nationwide,2400
4508,2021-09-29,INT-11894497,Mass Tort,Elmiron,infra-intakes,Converted,0,Ask.Law,Elmiron,Nationwide,2400
3693,2021-10-01,INT-11835660,Mass Tort,Elmiron,infra-intakes,Converted,0,Ask.Law,Elmiron,Nationwide,2400
5379,2021-10-01,INT-11901586,Mass Tort,Elmiron,infra-intakes,Converted,0,Ask.Law,Elmiron,Nationwide,2400
3290,2021-10-06,INT-11860814,Mass Tort,Elmiron,infra-intakes,Converted,0,Ask.Law,Elmiron,Nationwide,2400
391,2021-10-07,INT-11909923,Mass Tort,Elmiron,infra-intakes,Converted,0,Ask.Law,Elmiron,Nationwide,2400
3808,2021-10-09,INT-11804986,Mass Tort,Elmiron,infra-intakes,Converted,0,Ask.Law,Elmiron,Nationwide,2400
1783,2021-10-09,INT-11906663,Mass Tort,Elmiron,infra-intakes,Converted,0,Ask.Law,Elmiron,Nationwide,2400


In [33]:
# filtering the rows based on the casetype
retained_intakes.loc[retained_intakes.casetype.str.contains('T-Mobile Data Breach', case=False, na=False)]

,created_date,intake,litigation,casetype,input_channel,status,rosu,marketing_source,marketing_subsource,venue,value
2678,2021-11-04,INT-11704711,Mass Arbitration,T-Mobile Data Breach,Bird Web Questionnaire,Converted,0,BIRD,Ask Law,Nationwide,20
4177,2021-11-04,INT-11712359,Mass Arbitration,T-Mobile Data Breach,Bird Web Questionnaire,Converted,0,BIRD,Ask Law,Nationwide,20
4311,2021-11-04,INT-11721517,Mass Arbitration,T-Mobile Data Breach,Bird Web Questionnaire,Converted,0,BIRD,Ask Law,Nationwide,20
5486,2021-11-04,INT-11729568,Mass Arbitration,T-Mobile Data Breach,Bird Web Questionnaire,Converted,0,BIRD,Ask Law,Nationwide,20
3370,2021-11-04,INT-11747752,Mass Arbitration,T-Mobile Data Breach,Bird Web Questionnaire,Converted,0,BIRD,Ask Law,Nationwide,20
...,...,...,...,...,...,...,...,...,...,...,...
3885,2021-11-29,INT-11969965,Mass Arbitration,T-Mobile Data Breach,Bird Web Questionnaire,Retainer Received,0,BIRD,Ask Law,Nationwide,20
5406,2021-11-29,INT-11972830,Mass Arbitration,T-Mobile Data Breach,Bird Web Questionnaire,Retainer Received,0,BIRD,Ask Law,Nationwide,20
1999,2021-11-29,INT-11980375,Mass Arbitration,T-Mobile Data Breach,Bird Web Questionnaire,Retainer Received,0,BIRD,Ask Law,Nationwide,20
1962,2021-11-29,INT-11991048,Mass Arbitration,T-Mobile Data Breach,Bird Web Questionnaire,Retainer Received,0,BIRD,Ask Law,Nationwide,20


In [35]:
# Preparing to sum up the billing by venue litigation casetype and month by creating a month column using created_date
retained_intakes['month'] = retained_intakes.created_date.dt.month

groupby_fields = ['venue', 'litigation', 'casetype', 'month']
retained_intakes.groupby(groupby_fields).value.sum()

venue            litigation          casetype             month
AL - Birmingham  Personal Injury     Automobile Accident  11       3000
                 Premises Liability  Slip and Fall        7        1000
                                                          11       1000
AL - Montgomery  Personal Injury     Automobile Accident  11       1000
AZ - Phoenix     Personal Injury     Automobile Accident  5        4000
                                                                   ... 
PA - Pittsburgh  Personal Injury     Automobile Accident  10       1000
                 Premises Liability  General Injury       8        1000
                                     Slip and Fall        9        1000
SC - ATL         Personal Injury     Automobile Accident  10       1000
TN - Memphis     Personal Injury     Automobile Accident  7        1000
Name: value, Length: 112, dtype: int64

In [54]:
# creating a second month column and converting it to string because only month was being displayed which will create problems with book keeping for Financial Analyst
retained_intakes['month2'] = retained_intakes['created_date'].dt.strftime('%b %y')

In [57]:
retained_intakes.dtypes

created_date           datetime64[ns]
intake                         object
litigation                     object
casetype                       object
input_channel                  object
status                         object
rosu                            int64
marketing_source               object
marketing_subsource            object
venue                          object
value                           int64
month                           int64
month2                         object
year                            int32
month_year                      int64
dtype: object

In [59]:
# Extract the month and year as separate columns
retained_intakes['month'] = retained_intakes['month2'].str[:3]
retained_intakes['year'] = retained_intakes['month2'].str[-2:].astype(int)

# Combine the month and year into a single integer column
retained_intakes['month_year'] = retained_intakes['year'] * 100 + retained_intakes['month'].map({
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
})

# Convert the 'month_year' values back to the "Abbreviated Month Year" format
retained_intakes['month2'] = retained_intakes['month'].str[:] + ' ' + retained_intakes['year'].astype(str).str[-2:]


In [62]:
retained_intakes.pivot_table(index=['venue', 'litigation', 'casetype'],
                             values=['value'],
                             columns=['month_year'],
                             aggfunc=np.sum,
                             fill_value='-'
                             )

value           
month_year                                                2105    2106   
venue           litigation         casetype                              
AL - Birmingham Personal Injury    Automobile Accident       -       -  \
                Premises Liability Slip and Fall             -       -   
AL - Montgomery Personal Injury    Automobile Accident       -       -   
AZ - Phoenix    Personal Injury    Automobile Accident  4000.0  3000.0   
                Premises Liability General Injury       2000.0       -   
...                                                        ...     ...   
PA - Pittsburgh Personal Injury    Automobile Accident       -       -   
                Premises Liability General Injury            -       -   
                                   Slip and Fall             -       -   
SC - ATL        Personal Injury    Automobile Accident       -       -   
TN - Memphis    Personal Injury    Automobile Accident       -       -   

                                                                         
month_year                                                2107    2108   
venue           litigation         casetype                              
AL - Birmingham Personal Injury    Automobile Accident       -       -  \
                Premises Liability Slip and Fall        1000.0       -   
AL - Montgomery Personal Injury    Automobile Accident       -       -   
AZ - Phoenix    Personal Injury    Automobile Accident  4000.0  4000.0   
                Premises Liability General Injury       1000.0  1000.0   
...                                                        ...     ...   
PA - Pittsburgh Personal Injury    Automobile Accident       -       -   
                Premises Liability General Injury            -  1000.0   
                                   Slip and Fall             -       -   
SC - ATL        Personal Injury    Automobile Accident       -       -   
TN - Memphis    Personal Injury    Automobile Accident  1000.0       -   

                                                                                
month_year                                                2109    2110    2111  
venue           litigation         casetype                                     
AL - Birmingham Personal Injury    Automobile Accident       -       -  3000.0  
                Premises Liability Slip and Fall             -       -  1000.0  
AL - Montgomery Personal Injury    Automobile Accident       -       -  1000.0  
AZ - Phoenix    Personal Injury    Automobile Accident  7000.0  9000.0  2000.0  
                Premises Liability General Injury       1000.0       -  1000.0  
...                                                        ...     ...     ...  
PA - Pittsburgh Personal Injury    Automobile Accident  1000.0  1000.0       -  
                Premises Liability General Injury            -       -       -  
                                   Slip and Fall        1000.0       -       -  
SC - ATL        Personal Injury    Automobile Accident       -  1000.0       -  
TN - Memphis    Personal Injury    Automobile Accident       -       -       -  

[62 rows x 7 columns]

In [63]:
# Saving our manipulated dataframe 
retained_intakes.to_csv('retained_intakes.csv', index=False)
